In [1]:
import numpy as np
import pandas as pd
import keras
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, GlobalMaxPooling2D, GlobalAveragePooling2D
from keras.layers import Dense, Dropout, Activation, Embedding, Flatten, Reshape
from keras.utils import np_utils
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.pipeline import Pipeline

Using TensorFlow backend.


In [2]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import TruncatedSVD
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import Normalizer, StandardScaler, LabelEncoder
from sklearn.metrics import classification_report, accuracy_score, confusion_matrix
from sklearn.model_selection import train_test_split

In [3]:
import nltk
from nltk import word_tokenize
from nltk.stem import WordNetLemmatizer

In [4]:
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

# Load data from file

In [5]:
merged_data = pd.read_pickle('merged_data_pct_change.pkl')

In [6]:
filtered_merged_data = merged_data[merged_data['1day pct change'].abs() > 3.]

In [7]:
len(filtered_merged_data)

1093

# Create train data and test data

In [8]:
X = filtered_merged_data['rawText']
y = filtered_merged_data['1day pct change'] > 0
y = np_utils.to_categorical(y, 2)

In [9]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

# Tf-Idf

In [10]:
# not used
class LemmaTokenizer(object):
    def __init__(self):
        self.wnl = WordNetLemmatizer()
    def __call__(self, doc):
        return [self.wnl.lemmatize(t.lower(), 'v') for t in word_tokenize(doc)]

In [11]:
tfidf = TfidfVectorizer(max_df=0.5, min_df=50, ngram_range=(1,2), stop_words='english')
svd = TruncatedSVD(2000)
#lsa = make_pipeline(tfidf, svd)
lsa = make_pipeline(tfidf)

# Run SVD on the training data, then project the training data.
train_vectors = lsa.fit_transform(X_train)
test_vectors = lsa.transform(X_test)

In [12]:
train_vectors.shape

(874, 6155)

In [13]:
train_vectors = train_vectors.toarray().reshape((train_vectors.shape[0], 1, train_vectors.shape[1], 1))
test_vectors = test_vectors.toarray().reshape((test_vectors.shape[0], 1, test_vectors.shape[1], 1))

In [14]:
train_vectors.shape

(874, 1, 6155, 1)

In [15]:
y_train.shape

(874, 2)

# Model

In [20]:
cnn = Sequential()
cnn.add(Conv2D(16, (2, 1), padding="same", activation="relu", input_shape=(1, train_vectors.shape[2], 1)))
#cnn.add(Conv2D(64, (10, 1), padding="same", activation="relu"))
cnn.add(MaxPooling2D(pool_size=(1,4)))

#cnn.add(Conv2D(128, (8, 1), padding="same", activation="relu"))
#cnn.add(Conv2D(128, (8, 1), padding="same", activation="relu"))
#cnn.add(Conv2D(128, (8, 1), padding="same", activation="relu"))
#cnn.add(MaxPooling2D(pool_size=(1,5)))
    
cnn.add(Conv2D(16, (3, 1), padding="same", activation="relu"))
cnn.add(Conv2D(32, (4, 1), padding="same", activation="relu"))
cnn.add(Conv2D(64, (5, 1), padding="same", activation="relu"))
cnn.add(MaxPooling2D(pool_size=(1,8)))
    
cnn.add(Flatten())
cnn.add(Dense(512, activation='relu'))
cnn.add(Dense(2, activation='softmax'))

cnn.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
cnn.fit(train_vectors, y_train, epochs=30, verbose=2)

Epoch 1/40
6s - loss: 0.6937 - acc: 0.5011
Epoch 2/40
4s - loss: 0.6934 - acc: 0.5149
Epoch 3/40
4s - loss: 0.6933 - acc: 0.5149
Epoch 4/40
4s - loss: 0.6930 - acc: 0.5149
Epoch 5/40
4s - loss: 0.6929 - acc: 0.5149
Epoch 6/40
4s - loss: 0.6929 - acc: 0.5149
Epoch 7/40
4s - loss: 0.6928 - acc: 0.5149
Epoch 8/40
4s - loss: 0.6930 - acc: 0.5172
Epoch 9/40
4s - loss: 0.6906 - acc: 0.5149
Epoch 10/40
4s - loss: 0.6809 - acc: 0.5755
Epoch 11/40
4s - loss: 0.6493 - acc: 0.6236
Epoch 12/40
4s - loss: 0.6168 - acc: 0.6533
Epoch 13/40
4s - loss: 0.5488 - acc: 0.7037
Epoch 14/40
4s - loss: 0.4226 - acc: 0.8032
Epoch 15/40
4s - loss: 0.2437 - acc: 0.8982
Epoch 16/40
4s - loss: 0.1018 - acc: 0.9680
Epoch 17/40
4s - loss: 0.0506 - acc: 0.9863
Epoch 18/40
4s - loss: 0.0401 - acc: 0.9897
Epoch 19/40
4s - loss: 0.0292 - acc: 0.9908
Epoch 20/40
4s - loss: 0.0101 - acc: 0.9977
Epoch 21/40
4s - loss: 0.0031 - acc: 1.0000
Epoch 22/40
4s - loss: 0.0028 - acc: 1.0000
Epoch 23/40
4s - loss: 0.0012 - acc: 1.00

# Prediction

In [21]:
y_predicted = cnn.predict_classes(test_vectors)

219/219 [==============================] - 0s     


In [22]:
y_test = y_test[:,1].astype(int)

IndexError: too many indices for array

In [23]:
print(classification_report(y_test, y_predicted))

             precision    recall  f1-score   support

          0       0.52      0.53      0.52       112
          1       0.50      0.49      0.49       107

avg / total       0.51      0.51      0.51       219

